# Lab 6

Daniel Alfredo Rayo Roldan

Gerardo Gabriel Pineda Riveiro

https://github.com/Gerax5/DS-lab6

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import string
from collections import Counter
import nltk
from nltk.corpus import stopwords as nltk_stop
from nltk.util import ngrams
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
import warnings
from textblob import TextBlob
import requests
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (classification_report, confusion_matrix, 
                           accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, roc_curve)
from sklearn.pipeline import Pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import json

warnings.filterwarnings('ignore')

## Cargado de Datos

Para este reporte se trabajó con un junton de datos igual a , que consistian 7613 en mensajes que tenian representaban mensajes positivos o negativos, las frases estaban escritas de un modo que solo usando el contexto y con conocimiento de la semantíca se podía entender si una frase representaba una catastrofe real o solamente figurada.

In [4]:
tweets = []

with open("data/traficogt.txt", "r", encoding="utf-16") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            tweet = json.loads(line)
            tweets.append(tweet)
        except json.JSONDecodeError as e:
            print("⚠️ Error parseando línea:", e)
            print("Línea problemática:", line[:200], "...")
            continue

print(f"Se cargaron {len(tweets)} tweets")


⚠️ Error parseando línea: Expecting ',' delimiter: line 1 column 4314 (char 4313)
Línea problemática: {"id": 1711127305131631071, "id_str": "1711127305131631071", "url": "https://x.com/nquevedoa/status/1711127305131631071", "date": "2023-10-08 21:11:55+00:00", "user": {"id": 204996481, "id_str": "2049 ...
Se cargaron 5604 tweets


In [5]:
first_rt = None
for i, tweet in enumerate(tweets):
    if tweet.get("retweetedTweet") is not None:
        first_rt = (i, tweet)
        break

if first_rt:
    idx, tw = first_rt
    print(f"El primer retweet está en tweets[{idx}]")
    print("ID:", tw["id"])
    print("Usuario:", tw["user"]["username"])
    print("Texto:", tw["rawContent"])
else:
    print("No se encontró ningún retweet")


No se encontró ningún retweet


In [6]:
print(tweets[0])

{'id': 1834236045598056867, 'id_str': '1834236045598056867', 'url': 'https://x.com/traficogt/status/1834236045598056867', 'date': '2024-09-12 14:22:06+00:00', 'user': {'id': 93938886, 'id_str': '93938886', 'url': 'https://x.com/traficogt', 'username': 'traficogt', 'displayname': 'traficoGT', 'rawDescription': 'Noticias de ciudad de Guatemala', 'created': '2009-12-01 20:42:19+00:00', 'followersCount': 314368, 'friendsCount': 137, 'statusesCount': 52385, 'favouritesCount': 3471, 'listedCount': 291, 'mediaCount': 1292, 'location': 'Guatemala', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1782036597841530880/-tVuhOdK_normal.jpg', 'profileBannerUrl': None, 'protected': None, 'verified': False, 'blue': False, 'blueType': None, 'descriptionLinks': [], 'pinnedIds': [], '_type': 'snscrape.modules.twitter.User'}, 'lang': 'es', 'rawContent': 'Es comprensible la resolución... El ruso sabe de engrasar maquinaria.', 'replyCount': 0, 'retweetCount': 0, 'likeCount': 1, 'quoteCount': 0, 'bo

In [ ]:
cleaned_tweets = []
for i, tweet in enumerate(tweets):
    value = {
        "id": tweet['id_str'],
        "content": tweet['rawContent'],
        "likeCount": tweet['likeCount'],
        "quoteCount": tweet['quoteCount'],
        "hashtags": tweet['hashtags'],
        "mentionedUsers": tweet['mentionedUsers'],
        "viewCount": tweet['viewCount'],
        "quotedTweetUser": tweet['quotedTweet']['id_str'],
        "quotedTweetId": tweet['quotedTweet'],
    }
    if tweet['quotedTweet'] is not None:
        value["quotedTweedUser"] = tweet['quotedTweet']['id_str']

    cleaned_tweets.append(value)
print(cleaned_tweets[0])

TypeError: 'NoneType' object is not subscriptable

### Para llevarnos

# user
* Username
* id_str
* followersCount
* friendsCount
* mediaCount
* verified
# rawContent
# likeCount
# quoteCount
# hashtags
# cashtags
# mentionedUsers
# viewCount
# retweetedTweet
# quotedTweet -> id


---
# quotedTweet
# user
* Username
* id_str
* followersCount
* friendsCount
* mediaCount
* verified
# rawContent
# likeCount
# quoteCount
# hashtags
# cashtags
# mentionedUsers
# viewCount
# retweetedTweet
# quotedTweet

# Analísis de sentimiento